In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import pickle
from sklearn.model_selection import cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB


In [2]:
# pip install streamlit

In [3]:
df = pd.read_csv(r'C:\Users\mayan\Downloads\Kaggle Data\heart.csv')

In [4]:
df

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


In [5]:
df.isnull().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [7]:
df.nunique()

Age                50
Sex                 2
ChestPainType       4
RestingBP          67
Cholesterol       222
FastingBS           2
RestingECG          3
MaxHR             119
ExerciseAngina      2
Oldpeak            53
ST_Slope            3
HeartDisease        2
dtype: int64

In [8]:
df["ST_Slope"].value_counts()

Flat    460
Up      395
Down     63
Name: ST_Slope, dtype: int64

In [9]:
df['RestingBP'].value_counts()

120    132
130    118
140    107
110     58
150     55
      ... 
113      1
117      1
127      1
129      1
0        1
Name: RestingBP, Length: 67, dtype: int64

In [10]:
df

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


In [11]:
df['FastingBS'].replace(to_replace=0,value='No',inplace=True)
df['FastingBS'].replace(to_replace=1,value='Yes',inplace=True)

df

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,No,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,No,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,No,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,No,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,No,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,No,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,Yes,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,No,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,No,LVH,174,N,0.0,Flat,1


In [12]:
df['ExerciseAngina'].value_counts()

N    547
Y    371
Name: ExerciseAngina, dtype: int64

In [13]:
df['RestingBP']

0      140
1      160
2      130
3      138
4      150
      ... 
913    110
914    144
915    130
916    130
917    138
Name: RestingBP, Length: 918, dtype: int64

In [14]:
df['Cholesterol'].value_counts()

0      172
254     11
223     10
220     10
211      9
      ... 
117      1
123      1
131      1
293      1
603      1
Name: Cholesterol, Length: 222, dtype: int64

In [15]:
df['MaxHR'].value_counts()

150    43
140    41
120    36
130    33
160    25
       ..
71      1
70      1
69      1
67      1
202     1
Name: MaxHR, Length: 119, dtype: int64

In [16]:
outliers = []

def detect_outliers(data):
    
    threshold = 3
    mean = np.mean(data)
    std = np.std(data)
    
    for i in data:
        z_score = (i-mean)/std
        if np.abs(z_score) > threshold:
            outliers.append(i)


detect_outliers(df['RestingBP'])


In [17]:
outliers

[190, 200, 200, 200, 0, 190, 200, 192]

In [18]:
index = []
for x in outliers:
    y = np.where(df['RestingBP']==x)
    index.append(y)
index

[(array([109, 592], dtype=int64),),
 (array([241, 365, 399, 732], dtype=int64),),
 (array([241, 365, 399, 732], dtype=int64),),
 (array([241, 365, 399, 732], dtype=int64),),
 (array([449], dtype=int64),),
 (array([109, 592], dtype=int64),),
 (array([241, 365, 399, 732], dtype=int64),),
 (array([759], dtype=int64),)]

In [19]:
df.drop([109,592,241,365,399,732,449,759],axis=0,inplace=True)

In [20]:
outliers = []

def detect_outliers(data):
    
    threshold = 3
    mean = np.mean(data)
    std = np.std(data)
    
    for i in data:
        z_score = (i-mean)/std
        if np.abs(z_score) > threshold:
            outliers.append(i)


detect_outliers(df['Cholesterol'])
outliers

[529, 603, 564]

In [21]:
index = []
for x in outliers:
    y = np.where(df['Cholesterol']==x)
    index.append(y)
index

[(array([76], dtype=int64),),
 (array([148], dtype=int64),),
 (array([610], dtype=int64),)]

In [22]:
df.drop([76,149,616],axis=0,inplace=True)

In [23]:
outliers = []

def detect_outliers(data):
    
    threshold = 3
    mean = np.mean(data)
    std = np.std(data)
    
    for i in data:
        z_score = (i-mean)/std
        if np.abs(z_score) > threshold:
            outliers.append(i)


detect_outliers(df['MaxHR'])
outliers

[60]

In [24]:
index = []
for x in outliers:
    y = np.where(df['Age']==x)
    index.append(y)
index

[(array([ 18, 276, 296, 307, 325, 326, 365, 375, 412, 417, 418, 419, 429,
         437, 445, 446, 466, 487, 503, 587, 614, 663, 669, 713, 718, 734,
         806, 810, 849, 852, 853, 865], dtype=int64),)]

In [25]:
df.drop([18, 276, 296, 307, 325, 326, 375, 412, 417, 418, 419, 429,
         437, 445, 446, 466, 487, 503, 587, 614, 663, 669, 713, 718, 734,
         806, 810, 849, 852, 853, 865],axis=0,inplace=True)

In [26]:
X= df.drop(['HeartDisease'],axis=1)
y = df['HeartDisease']

In [27]:
cltf = ColumnTransformer([
    ('ohe',OneHotEncoder(drop='first',sparse=False),['Sex','ChestPainType','RestingECG','ExerciseAngina','ST_Slope'])],
      remainder='passthrough')

In [28]:
pipe = Pipeline(steps=[('step1',cltf),
                       ('step2',GaussianNB())])

In [29]:
pipe.fit(X,y)

ValueError: could not convert string to float: 'No'

In [ ]:
cross_val_score(estimator=pipe,X=X,y=y,scoring='accuracy',cv=5)

In [ ]:
import pickle

In [ ]:
pickle.dump(pipe,open('pipe.pkl','wb'))